## IRS

   - An IRS is a financial derivative contract in which two parties exchange cash flows of interest payments.
   - Typically, one party pays a fixed rate (fixed leg), while the other party pays a floating rate (floating leg).
   - The value of an IRS can be determined using the present value of the fixed and floating legs.

## Caplets and Floorlets

   - Caplets and floorlets are components of interest rate caps and floors, respectively. A caplet is a financial derivative that pays off when the interest rate exceeds a predetermined strike rate, whereas a floorlet pays off when the interest rate falls below a predetermined strike rate.
   - The formulas for caplets and floorlets involve several variables:
     - $ P(t, T_{i-1}) $: Discount factor from time $ t $ to $ T_{i-1} $
     - $ K $: Strike rate
     - $ Z_{T_i, T_{i-1}}^t $: Forward rate observed at time $ t $ for the period from $ T_{i-1} $ to $ T_i $
     - $ v(t) $: Volatility term
     - $ \Phi $: Cumulative distribution function (CDF) of the standard normal distribution
     - $ d_1 $ and $ d_2 $ are auxiliary variables defined as:
       
       $d_1(K, F, v) = \frac{\log(F/K) + \frac{v^2}{2}}{v}$
       
       
       $d_2(K, F, v) = d_1(K, F, v) - v$

## Implementation Steps

   - **Step 1: Calculate the discount factor $ P(t, T_{i-1})$ using the Hull-White model.**
   - **Step 2: Calculate the forward rate $Z_{T_i, T_{i-1}}^t $.**
   - **Step 3: Determine the volatility term $v(t) $.**
   - **Step 4: Compute the  $d_1$  and $d_2 $ variables.**
   - **Step 5: Compute the put and call price.**
   - **Step 6: Calculate the caplet and floorlet prices using the provided formulas.**

In [207]:
from scipy.stats import norm
import numpy as np

In [208]:
class factors():
    def __init__(self, t, K, alpha, sigma, theta, r):
        self.t = t
        self.K = K 
        self.alpha = alpha
        self.sigma= sigma
        self.theta = theta
        self.r = r

    def B(self, s, T):
        B = (1 - np.exp(-self.alpha * (T - s))) / self.alpha
        return B
    
    def A(self,T):
        A = np.exp((self.B(self.t, T) - (T - self.t)) * (self.theta - 0.5 * self.sigma ** 2 / self.alpha ** 2) - (self.sigma ** 2 * self.B(self.t, T) ** 2) / (4 * self.alpha))
        return A
    
    def discount_factor(self, T):
        B = self.B(self.t, T)
        A = self.A(T)
        return A * np.exp(-B * self.r) 
    
    def forward_rate(self, T1, T2):
        P_t_T1 = self.discount_factor(T1, T2)
        P_t_T2 = self.discount_factor()
        return (P_t_T1 / P_t_T2 - 1) / (T2 - T1)
    
    def Z_T1_T2(self, T1, T2):
        P_t_T1 = self.discount_factor(T1)
        P_t_T2 = self.discount_factor(T2)
        return P_t_T1 / P_t_T2   
    
    def d1_d2(self, F, v):
        d1 = (np.log(F / self.K) + 0.5 * v ** 2) / v
        d2 = d1 - v
        return d1, d2
    
    def volatility(self, T1, T2, dt):
        v = np.sqrt(np.sum([self.sigma**2 * (self.B(s, T1) - self.B(s, T2))**2 for s in np.arange(self.t, T2, dt)]))
        return v
    
    def K_star(self, delta):
        return 1 / (1 + delta * self.K)
    

- $PUT_{K,T_{i-1},T_{i}}(t) = P(t,T_{i-1})\left[K\Phi\left(-d_2(K,Z_{T_i,T_{i-1}}^t ,v(t))\right)-Z_{T_i,T_{i-1}}^t  \Phi \left(-d_1(K, Z_{T_i,T_{i-1}}^t, v(t))\right)\right]$

- $CALL_{K,T_{i-1},T_{i}}(t) = \frac{P(t,T_{i-1})}{K^{*}}\left[-K\Phi\left(-d_2(K,Z_{T_i,T_{i-1}}^t ,v(t))\right)+Z_{T_i,T_{i-1}}^t  \Phi \left(-d_1(K, Z_{T_i,T_{i-1}}^t, v(t))\right)\right]$
- $CAPLET(t)  = \frac{1}{K^*}*PUT_{K^*,T_{i-1},T_{i}}(t)$
- $FLOORLET(t)  = \frac{1}{K^*}*CALL_{K^*,T_{i-1},T_{i}}(t)$

 where $Z_{S,T}^t=\frac{P(t,S)}{P(t,T)}$, $K^* = \frac{1}{1+\delta*K}$

In [209]:
class Options(factors):
    def __init__(self, t, K, alpha, sigma, theta, r):
        super().__init__(t=t, K=K, alpha=alpha, sigma=sigma, theta=theta, r=r)
    
    def call_price(self, T1, T2, dt):
        P_t_T2 = super().discount_factor(T2)
        Z_T1_T2 = super().Z_T1_T2(T1, T2)
        v = super().volatility(T1, T2, dt)
        d1, d2 = super().d1_d2(Z_T1_T2, v)
        call_price = P_t_T2 * (Z_T1_T2 * norm.cdf(d1) - self.K * norm.cdf(d2))
        return call_price
    
    def put_price(self, T1, T2, dt):
        P_t_T2 = super().discount_factor(T2)
        Z_T1_T2 = super().Z_T1_T2(T1, T2)
        v = super().volatility(T1, T2, dt)
        d1, d2 = super().d1_d2(Z_T1_T2, v)
        put_price = P_t_T2 * (self.K * norm.cdf(-d2) - Z_T1_T2 * norm.cdf(-d1) )
        return put_price

In [210]:
class caplets_and_floorlet(Options):
    def __init__(self, t, K, alpha, sigma, theta, r):
        super().__init__(t=t, K=K, alpha=alpha, sigma=sigma, theta=theta, r=r)
        
    def caplet(self, T1, T2, dt):
        put_price = super().put_price(T1, T2, dt)
        caplet = (1 / self.K) * put_price
        return caplet
    
    def floorlet(self, T1, T2, dt):
        call_price = super().call_price(T1, T2, dt)
        floorlet = (1 / self.K) * call_price
        return floorlet

In [211]:
t = 0
T1 = 0.5
T2 = 1
alpha = 0.10
sigma = 0.09
theta = 0.2
r0 = 0.05
dt = 0.01
K = 1.25

delta = T2 -T1
K_star = factors(t = t, K = K, alpha = alpha, sigma = sigma, theta = theta, r = r0).K_star(delta = delta)
Option = Options(t = t, K = K, alpha = alpha, sigma = sigma, theta = theta, r = r0)
call_price = Option.call_price(T1=T1, T2 = T2, dt = dt)
put_price = Option.put_price(T1=T1, T2 = T2, dt = dt)

print(f"call price:{call_price}\nput price:{put_price}")

call price:0.10076650994774164
put price:0.30901586310688717


In [212]:
delta = T2 -T1
caplets_and_floorlet = caplets_and_floorlet(t = t, K = K_star, alpha=alpha, sigma = sigma, theta = theta, r = r0)
caplet = caplets_and_floorlet.caplet(T1 = T1, T2 = T2, dt = dt)
floorlet = caplets_and_floorlet.floorlet(T1 = T1, T2 = T2, dt = dt)

print(f"caplet:{caplet}\nfloorlet:{floorlet}")

caplet:0.031359355195693855
floorlet:0.668038644410333


Explanation
 1. **Discount Factor**: Calculated using a simplified version of the Hull-White model.
 2. **Forward Rate**: Derived from the ratio of discount factors.
 3. **Volatility**: Simplified to be constant for demonstration purposes.
 4. **d1 and d2**: Computed using the given formulas.
 5. **K^{*}**: Defined as $\frac1{1+ \delta K}$
 6. **Caplet and Floorlet Prices**: Derived using the provided formulas and the normal CDF.

 This implementation assumes simplified calculations for discount factors and volatility. In a real-world scenario, more complex methods and market data would be used. Adjustments might be necessary to fully align with the Hull-White model parameters and market conditions.